In [1]:
# 04_train_mlp_8neuron_metrics.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.metrics import classification_report, confusion_matrix

# === 1. Load dataset hasil windowing ===
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_val   = np.load("X_val.npy")
y_val   = np.load("y_val.npy")
X_test  = np.load("X_test.npy")
y_test  = np.load("y_test.npy")

print("Data shapes:")
print("  X_train:", X_train.shape)
print("  X_val  :", X_val.shape)
print("  X_test :", X_test.shape)

# === 2. Normalisasi (optional tapi disarankan) ===
X_train = X_train.astype("float32")
X_val   = X_val.astype("float32")
X_test  = X_test.astype("float32")

# === 3. Flatten input (karena MLP butuh 1D vector) ===
X_train = X_train.reshape(X_train.shape[0], -1)
X_val   = X_val.reshape(X_val.shape[0], -1)
X_test  = X_test.reshape(X_test.shape[0], -1)

input_dim = X_train.shape[1]
num_classes = len(np.unique(y_train))

print(f"Input dim: {input_dim}, Classes: {num_classes}")

# === 4. Bangun model MLP (1 hidden layer, 8 neuron) ===
model = models.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(8, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Optimizer Adam dengan lr 0.001
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# === 5. Siapkan callbacks ===
cb_earlystop = callbacks.EarlyStopping(
    monitor='val_loss', patience=30, restore_best_weights=True)

cb_checkpoint = callbacks.ModelCheckpoint(
    filepath='best_mlp8_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1)

callbacks_list = [cb_earlystop, cb_checkpoint]

# === 6. Latih model ===
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,             # biarkan besar, early stopping akan hentikan otomatis
    batch_size=32,
    callbacks=callbacks_list,
    verbose=1
)

# === 7. Evaluasi model terbaik di test set ===
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Akurasi test: {test_acc*100:.2f}%")

# === 8. Prediksi untuk metrik tambahan ===
y_pred = np.argmax(model.predict(X_test), axis=1)

# Buat laporan metrik klasifikasi
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

# Confusion matrix opsional
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# === 9. Simpan model final & versi TFLite ===
model.save("mlp_8neuron_final.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open("mlp_8neuron_model.tflite", "wb") as f:
    f.write(tflite_model)

print("\n✅ Model terbaik disimpan sebagai:")
print(" - best_mlp8_model.keras  (callback terbaik)")
print(" - mlp_8neuron_model.tflite (untuk ESP32)")


Data shapes:
  X_train: (1149, 50, 6)
  X_val  : (285, 50, 6)
  X_test : (357, 50, 6)
Input dim: 300, Classes: 3


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │         2,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,435 (9.51 KB)

 Trainable params: 2,435 (9.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
30/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3640 - loss: 25.0682   
Epoch 1: val_loss improved from inf to 28.10881, saving model to best_mlp8_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3571 - loss: 25.2636 - val_accuracy: 0.2807 - val_loss: 28.1088
Epoch 2/200
35/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3439 - loss: 18.8866 
Epoch 2: val_loss improved from 28.10881 to 22.18529, saving model to best_mlp8_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3442 - loss: 18.7687 - val_accuracy: 0.3018 - val_loss: 22.1853
Epoch 3/200
 1/36 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5312 - loss: 18.4538
Epoch 3: val_loss improved from 22.18529 to 17.72155, saving model to best_mlp8_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4273 - loss: 11.7243 - val_accuracy: 0.3895 - val_loss: 17.7215
Epoch 4/200
35/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5008 - loss: 6.9426 
Epoch 4: val_loss improved fr

INFO:tensorflow:Assets written to: C:\Users\muham\AppData\Local\Temp\tmp95nor38i\assets


Saved artifact at 'C:\Users\muham\AppData\Local\Temp\tmp95nor38i'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1588810983632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588810984400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588810982672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588810985552: TensorSpec(shape=(), dtype=tf.resource, name=None)

✅ Model terbaik disimpan sebagai:
 - best_mlp8_model.keras  (callback terbaik)
 - mlp_8neuron_model.tflite (untuk ESP32)


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.metrics import classification_report, confusion_matrix

# ===== 1) load windowed data =====
X_train = np.load("X_train.npy")  # (N, 50, 6)
y_train = np.load("y_train.npy")
X_val   = np.load("X_val.npy")
y_val   = np.load("y_val.npy")
X_test  = np.load("X_test.npy")
y_test  = np.load("y_test.npy")

print("Shapes:", X_train.shape, X_val.shape, X_test.shape)

# ===== 2) z-score per channel (pakai statistik TRAIN saja) =====
def fit_channel_stats(X):  # X:(N, T, C)
    XR = X.reshape(-1, X.shape[-1])     # (N*T, C)
    mean = XR.mean(axis=0)
    std  = XR.std(axis=0) + 1e-8
    return mean, std

def apply_channel_stats(X, mean, std):
    return (X - mean) / std

mean_tr, std_tr = fit_channel_stats(X_train)
np.savez("scaler_stats.npz", mean=mean_tr, std=std_tr)
print("Train means :", np.round(mean_tr, 4))
print("Train stds  :", np.round(std_tr, 4))

X_train = apply_channel_stats(X_train, mean_tr, std_tr).astype("float32")
X_val   = apply_channel_stats(X_val,   mean_tr, std_tr).astype("float32")
X_test  = apply_channel_stats(X_test,  mean_tr, std_tr).astype("float32")

# ===== 3) flatten untuk MLP =====
X_train = X_train.reshape(X_train.shape[0], -1)  # (N, 300)
X_val   = X_val.reshape(X_val.shape[0], -1)
X_test  = X_test.reshape(X_test.shape[0], -1)

input_dim   = X_train.shape[1]
num_classes = len(np.unique(y_train))
print(f"Input dim={input_dim}, classes={num_classes}")

# ===== 4) model: 1 hidden layer (8) + BatchNorm opsional =====
inputs = layers.Input(shape=(input_dim,))
x = layers.BatchNormalization()(inputs)      # opsional tapi membantu
x = layers.Dense(8, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

opt = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# ===== 5) callbacks =====
cb_early = callbacks.EarlyStopping(
    monitor='val_loss', patience=30, restore_best_weights=True)
cb_ckpt = callbacks.ModelCheckpoint(
    filepath='best_mlp8_model.keras',
    monitor='val_loss', save_best_only=True, verbose=1)

# ===== 6) train =====
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=300, batch_size=32,
    callbacks=[cb_early, cb_ckpt],
    verbose=1
)

# ===== 7) evaluate =====
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test accuracy: {test_acc*100:.2f}%")

y_pred = np.argmax(model.predict(X_test), axis=1)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion matrix:\n", cm)

# ===== 8) save final + tflite =====
model.save("mlp_8neuron_final.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]   # quantization-ready
tflite = converter.convert()
open("mlp_8neuron_model.tflite", "wb").write(tflite)
print("\nFiles saved:\n - best_mlp8_model.keras\n - mlp_8neuron_final.keras\n - mlp_8neuron_model.tflite\n - scaler_stats.npz")


Shapes: (1149, 50, 6) (285, 50, 6) (357, 50, 6)
Train means : [ 0.2645  0.0078  1.0119 -3.4102  0.6024 -2.5877]
Train stds  : [ 0.5355  0.3022  0.5587 35.4983 64.2229 41.9884]
Input dim=300, classes=3


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 300)            │         1,200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │         2,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,635 (14.20 KB)

 Trainable params: 3,035 (11.86 KB)

 Non-trainable params: 600 (2.34 KB)

Epoch 1/300
 1/36 ━━━━━━━━━━━━━━━━━━━━ 27s 799ms/step - accuracy: 0.4688 - loss: 1.4069
Epoch 1: val_loss improved from inf to 1.17322, saving model to best_mlp8_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3737 - loss: 1.3789 - val_accuracy: 0.5228 - val_loss: 1.1732
Epoch 2/300
 1/36 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3750 - loss: 1.1533
Epoch 2: val_loss improved from 1.17322 to 0.93819, saving model to best_mlp8_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5602 - loss: 0.9929 - val_accuracy: 0.6035 - val_loss: 0.9382
Epoch 3/300
 1/36 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8125 - loss: 0.6339
Epoch 3: val_loss improved from 0.93819 to 0.79282, saving model to best_mlp8_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6961 - loss: 0.7379 - val_accuracy: 0.6561 - val_loss: 0.7928
Epoch 4/300
 1/36 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.6875 - loss: 0.6241
Epoch 4: val_loss improved from 0.79282 to 

INFO:tensorflow:Assets written to: C:\Users\muham\AppData\Local\Temp\tmpho7p95fl\assets


Saved artifact at 'C:\Users\muham\AppData\Local\Temp\tmpho7p95fl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300), dtype=tf.float32, name='keras_tensor_3')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1588813299792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813301520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813310352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813300560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813300368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813310544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813314768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813313424: TensorSpec(shape=(), dtype=tf.resource, name=None)

Files saved:
 - best_mlp8_model.keras
 - mlp_8neuron_final.keras
 - mlp_8neuron_model.tflite
 - scaler_stats.npz


In [3]:
# 04_train_mlp_2layer16_metrics_norm.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.metrics import classification_report, confusion_matrix

# ===== 1) Load windowed data =====
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_val   = np.load("X_val.npy")
y_val   = np.load("y_val.npy")
X_test  = np.load("X_test.npy")
y_test  = np.load("y_test.npy")

print("Shapes:", X_train.shape, X_val.shape, X_test.shape)

# ===== 2) Z-score per channel (pakai statistik TRAIN saja) =====
def fit_channel_stats(X):
    XR = X.reshape(-1, X.shape[-1])  # (N*T, C)
    mean = XR.mean(axis=0)
    std = XR.std(axis=0) + 1e-8
    return mean, std

def apply_channel_stats(X, mean, std):
    return (X - mean) / std

mean_tr, std_tr = fit_channel_stats(X_train)
np.savez("scaler_stats.npz", mean=mean_tr, std=std_tr)
print("Train means :", np.round(mean_tr, 4))
print("Train stds  :", np.round(std_tr, 4))

X_train = apply_channel_stats(X_train, mean_tr, std_tr).astype("float32")
X_val   = apply_channel_stats(X_val,   mean_tr, std_tr).astype("float32")
X_test  = apply_channel_stats(X_test,  mean_tr, std_tr).astype("float32")

# ===== 3) Flatten untuk MLP =====
X_train = X_train.reshape(X_train.shape[0], -1)
X_val   = X_val.reshape(X_val.shape[0], -1)
X_test  = X_test.reshape(X_test.shape[0], -1)

input_dim = X_train.shape[1]
num_classes = len(np.unique(y_train))
print(f"Input dim={input_dim}, classes={num_classes}")

# ===== 4) Model: 2 hidden layer, 16 neuron per layer =====
inputs = layers.Input(shape=(input_dim,))
x = layers.BatchNormalization()(inputs)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

# ===== 5) Compile =====
opt = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ===== 6) Callbacks =====
cb_early = callbacks.EarlyStopping(
    monitor='val_loss', patience=30, restore_best_weights=True)

cb_ckpt = callbacks.ModelCheckpoint(
    filepath='best_mlp2x16_model.keras',
    monitor='val_loss', save_best_only=True, verbose=1)

# ===== 7) Training =====
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    callbacks=[cb_early, cb_ckpt],
    verbose=1
)

# ===== 8) Evaluation =====
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test accuracy: {test_acc*100:.2f}%")

y_pred = np.argmax(model.predict(X_test), axis=1)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion matrix:\n", cm)

# ===== 9) Save final model + TFLite =====
model.save("mlp_2layer16_final.keras")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]   # quantization-ready
tflite_model = converter.convert()
open("mlp_2layer16_model.tflite", "wb").write(tflite_model)

print("\nFiles saved:")
print(" - best_mlp2x16_model.keras  (callback best)")
print(" - mlp_2layer16_final.keras  (final model)")
print(" - mlp_2layer16_model.tflite (for ESP32)")
print(" - scaler_stats.npz          (normalization stats)")


Shapes: (1149, 50, 6) (285, 50, 6) (357, 50, 6)
Train means : [ 0.2645  0.0078  1.0119 -3.4102  0.6024 -2.5877]
Train stds  : [ 0.5355  0.3022  0.5587 35.4983 64.2229 41.9884]
Input dim=300, classes=3


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 300)            │         1,200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │         4,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,339 (24.76 KB)

 Trainable params: 5,739 (22.42 KB)

 Non-trainable params: 600 (2.34 KB)

Epoch 1/200
34/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4038 - loss: 1.1811   
Epoch 1: val_loss improved from inf to 0.90046, saving model to best_mlp2x16_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4083 - loss: 1.1723 - val_accuracy: 0.6526 - val_loss: 0.9005
Epoch 2/200
 1/36 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5938 - loss: 0.8755
Epoch 2: val_loss improved from 0.90046 to 0.71494, saving model to best_mlp2x16_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6361 - loss: 0.8414 - val_accuracy: 0.7053 - val_loss: 0.7149
Epoch 3/200
35/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8048 - loss: 0.6158 
Epoch 3: val_loss improved from 0.71494 to 0.57313, saving model to best_mlp2x16_model.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8036 - loss: 0.6148 - val_accuracy: 0.7509 - val_loss: 0.5731
Epoch 4/200
 1/36 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7500 - loss: 0.5311
Epoch 4: val_loss improved from 0.

INFO:tensorflow:Assets written to: C:\Users\muham\AppData\Local\Temp\tmpifnxbyrk\assets


Saved artifact at 'C:\Users\muham\AppData\Local\Temp\tmpifnxbyrk'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300), dtype=tf.float32, name='keras_tensor_7')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1588813733840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813737872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813734416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813735952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813736912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813732688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813735184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813728080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813738064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1588813729808: TensorSpec(shape=(), dtype=tf.resource, name=None)

Files saved: